# PyLASCA
a tool for ...

## Area Analysis
Analyse and compare areas of LaSCA and mechanical pain detection 

by

Sebastian Malkusch

2020-06-16

Institute for clinical pharmacology,

data science division,

Goethe-University Frankfurt,

Germany

malkusch@med.uni-frankfurt.de

www.smb.uni-frankfurt.de

http://share.smb.uni-frankfurt.de

## Load Libraries

In [1]:
import cv2 as cv
import pandas as pd
import numpy as np
from os import walk
import os.path
import re
from pyLASCA.source import fileNames
from pyLASCA.source import baseLine
#from pyLASCA.source import loss

## User Definitions

In [2]:
probandNr = 1
#effect = "allodynia"
effect = "sec_hyperalgesia"

## Browse Image Filenames

In [3]:
image_path = "/Users/malkusch/PowerFolders/LaSCA/rawData" 
image_path += str("/SCTCAPS%s" %(str(probandNr).zfill(2)))
fn_image = []
for (dirpath, dirnames, filenames) in walk(image_path):
    fn_image.extend(filenames)
    break

## Browse Coordinate Filenames

In [4]:
coordinate_main_folder = "/Users/malkusch/PowerFolders/LaSCA/mechanic"
coordinate_main_folder += str("/SCTCAPS %s" % (str(probandNr).zfill(2)))

## Init Dataframe

In [5]:
day_array = []
min_array = []
frame_array = []
for i in range (len(fn_image)):
    time_array = fn_image[i].split("_")[1].split(".")
    if (len(time_array) == 1):
        result = re.search(r'\d+', time_array[0])
        if(result):
            day_array.append(int(result.group()))
            min_array.append(-15)
            frame_array.append(int(re.search(r'\d+', fn_image[i].split("_")[2].split(".")[0]).group()))
    if (len(time_array) == 2):
        day_array.append(int(time_array[0]))
        min_array.append(int(time_array[1]))
        frame_array.append(int(re.search(r'\d+', fn_image[i].split("_")[2].split(".")[0]).group()))
id_array = np.zeros(len(day_array), dtype = int) + probandNr
effect_array = [effect]*len(day_array)

df = pd.DataFrame({"Id": id_array,
                  "day": day_array,
                  "time_[min]": min_array,
                   "frame": frame_array,
                   "effect": effect_array
                  })
df.head()

,Id,day,time_[min],frame,effect
0,1,2,60,1,sec_hyperalgesia
1,1,2,15,0,sec_hyperalgesia
2,1,1,-15,0,sec_hyperalgesia
3,1,2,45,2,sec_hyperalgesia
4,1,2,60,0,sec_hyperalgesia


## Pxl Size

In [6]:
pxl_size_filename = "/Users/malkusch/PowerFolders/LaSCA/Probanden Arm Bilder mit Markierung/pxl_size.xlsx"
df_pxl = pd.read_excel(io = pxl_size_filename, index_col = 0)
pxl_size_array = np.zeros(len(df), dtype = float)
for i in range(len(df)):
    temp_id = int(df["Id"].loc[i])
    temp_day = int(df["day"].loc[i])
    temp_time = int(df["time_[min]"].loc[i])
    pxl_size_array[i] = df_pxl[(df_pxl["object_id"] == temp_id) & (df_pxl["day"] == temp_day) & (df_pxl["time_min"] == temp_time)]["pxlSize_mm"].values[0]

df["pxlSize_[mm]"] = pxl_size_array
df.head()

,Id,day,time_[min],frame,effect,pxlSize_[mm]
0,1,2,60,1,sec_hyperalgesia,0.244449
1,1,2,15,0,sec_hyperalgesia,0.244449
2,1,1,-15,0,sec_hyperalgesia,0.240843
3,1,2,45,2,sec_hyperalgesia,0.244449
4,1,2,60,0,sec_hyperalgesia,0.244449


## Intensity Statistics

In [7]:
mean_gray_array = np.zeros(len(df), dtype = float)
median_gray_array = np.zeros(len(df), dtype = float)
std_gray_array = np.zeros(len(df), dtype = float)
min_gray_array = np.zeros(len(df), dtype = int)
max_gray_array = np.zeros(len(df), dtype = int)
bl = baseLine.BaseLine()
for i in range(len(df)):
    if (df["time_[min]"].loc[i] < 0):
        image_filename = image_path + str("/SCTCAPS%s_baseline%i_frame%i.tiff" % (str(df["Id"].loc[i]).zfill(2), df["day"].loc[i], df["frame"].loc[i]))
    else:
        image_filename = image_path + str("/SCTCAPS%s_%i.%i_frame%i.tiff" % (str(df["Id"].loc[i]).zfill(2), df["day"].loc[i], df["time_[min]"].loc[i] , df["frame"].loc[i])) 
    if not(os.path.exists(image_filename)):
        raise ValueError('could not find file %s.' % (image_filename))
    image_baseline = cv.imread(image_filename, cv.IMREAD_ANYDEPTH)

    bl.image = image_baseline
    bl.fit()
    mean_gray_array[i] = bl.mean_gray_value
    median_gray_array[i] = bl.median_gray_value
    std_gray_array[i] = bl.std_gray_value
    min_gray_array[i] = bl.min_gray_value
    max_gray_array[i] = bl.max_gray_value


df["median_[adu]"] = median_gray_array
df["mean_[adu]"] = mean_gray_array
df["std_[adu]"] = std_gray_array
df["min_[adu]"] = min_gray_array
df["max_[adu]"] = max_gray_array

df.head()

,Id,day,time_[min],frame,effect,pxlSize_[mm],median_[adu],mean_[adu],std_[adu],min_[adu],max_[adu]
0,1,2,60,1,sec_hyperalgesia,0.244449,35.0,48.287601,38.353185,6,261
1,1,2,15,0,sec_hyperalgesia,0.244449,37.0,51.190322,36.943903,9,247
2,1,1,-15,0,sec_hyperalgesia,0.240843,53.0,53.579980,14.698054,3,122
3,1,2,45,2,sec_hyperalgesia,0.244449,39.0,55.012654,43.852552,7,271
4,1,2,60,0,sec_hyperalgesia,0.244449,33.0,46.377319,37.599478,8,254


## Area Comparison

In [8]:
appl_array = ["none"]*len(df)
alpha_array = np.zeros(len(df), dtype = float)
thr_array = np.zeros(len(df), dtype = float)
mechanic_area_radius_array = np.zeros(len(df), dtype = float)
lasca_area_radius_array = np.zeros(len(df), dtype = float)
intersect_area_radius = np.zeros(len(df), dtype = float)
dist_array = np.zeros(len(df), dtype = float)
delta_overlay_radius_array = np.zeros(len(df), dtype = float)
delta_area_radius_array = np.zeros(len(df), dtype = float)
loss_array = np.zeros(len(df), dtype = float)

baseline_frame = 0
lf = loss.Loss()

for i in range(len(df)):
    if (df["time_[min]"].loc[i] < 0):
        alpha_array[i] = np.nan
        thr_array[i] = np.nan
        mechanic_area_radius_array[i] = np.nan
        lasca_area_radius_array[i] = np.nan
        intersect_area_radius[i] = np.nan
        dist_array[i] = np.nan
        delta_overlay_radius_array[i] = np.nan
        delta_area_radius_array[i] = np.nan
        loss_array[i] = np.nan
        continue
    else:
        image_filename = image_path + str("/SCTCAPS%s_%i.%i_frame%i.tiff" % (str(df["Id"].loc[i]).zfill(2), df["day"].loc[i], df["time_[min]"].loc[i] , df["frame"].loc[i])) 
        if not(os.path.exists(image_filename)):
            raise ValueError('could not find file %s.' % (image_filename))
        coordinate_sub_folder = coordinate_main_folder + str("/Tag %i" %(df["day"].loc[i]))
        coordinate_filename = coordinate_sub_folder + str("/SCTCAPS %s #%i %imin (Colour)_transformed_200612.csv" %(str(df["Id"].loc[i]).zfill(2), df["day"].loc[i], df["time_[min]"].loc[i]))
        if not(os.path.exists(coordinate_filename)):
            raise ValueError('could not find file %s.' % (coordinate_filename))
        df_coord = pd.read_csv(coordinate_filename)
        df_coord = df_coord[df_coord["effect"] == effect]
        contour_usr = np.zeros([len(df_coord),1,2], dtype = int)
        contour_usr[:,0,0] = np.round(df_coord["x_img_[pxl]"].values).astype(int)
        contour_usr[:,0,1] = np.round(df_coord["y_img_[pxl]"].values).astype(int)
        
        lf.pxl_size = df["pxlSize_[mm]"].loc[i]
        lf.median = df[(df["time_[min]"]==-15) & (df["day"]==df["day"].loc[i]) & (df["frame"] == 0)]["median_[adu]"].values[0]
        lf.sigma = df[(df["time_[min]"]==-15) & (df["day"]==df["day"].loc[i]) & (df["frame"] == 0)]["std_[adu]"].values[0]
        lf.image_raw = cv.imread(image_filename, cv.IMREAD_ANYDEPTH)
        lf.contour_usr = contour_usr
        lf.fit_brute()
        
        appl_array[i] = df_coord["appl_type"].values[0]
        alpha_array[i] = lf.alpha
        thr_array[i] = lf.thr_abs
        mechanic_area_radius_array[i] = lf.area_r_usr
        lasca_area_radius_array[i] = lf.area_r_thr
        intersect_area_radius[i] = lf.area_r_intersect
        dist_array[i] = lf.delta_dist
        delta_overlay_radius_array[i] = lf.delta_overlay_radius
        delta_area_radius_array[i] = lf.delta_area_radius
        loss_array[i] = lf.loss
        
df["apll_type"] = appl_array
df["alpha"] = alpha_array
df["thr_[adu]"] = thr_array
df["A_mech_r_[mm]"] = mechanic_area_radius_array
df["A_lasca_r_[mm]"] = lasca_area_radius_array
df["A_intersec_r_[mm]"] = intersect_area_radius
df["delta_dist_r_[mm]"] =  dist_array
df["delta_overlay_r_[mm]"] = delta_overlay_radius_array
df["delta_A_r_[mm]"] = delta_area_radius_array
df["loss_[mm]"] = loss_array

NameError: name 'loss' is not defined

In [ ]:
df[df["day"]==1].head()

## Log Results

In [ ]:
fn = fileNames.FileNames()
fn.folderName = "/Users/malkusch/PowerFolders/LaSCA/area"
fn.baseName = str("SCTCAPS%s" %(str(probandNr).zfill(2)))
fn.supplementalInformation = str("%s" %(effect))
fn.suffix = "csv"
fn.updateDateString()
fn.mergeFileName()
fn.outFileName

df.to_csv(path_or_buf = fn.outFileName)
print("wrote df to %s" %(fn.outFileName))